In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
glove_50_path = "/content/drive/MyDrive/Colab Notebooks/glove.6B.50d.txt"
glove_300_path = "/content/drive/MyDrive/Colab Notebooks/glove.6B.300d.txt"

## Loading Embedding

In [6]:
import progressbar
import numpy as np

def return_lines(path_to_txt):
  f = open(path_to_txt, "r", encoding="utf-8")
  lines = []
  i = 0
  while i<500000:
    try:
      line = f.readline()
      lines.append(line)
      i+=1
      if not line:
        break
    except:
      pass
  f.close()
  return lines

def emb_matrix_maker(path_to_txt):
  lines = return_lines(path_to_txt)
  bangla_glove_dim = {}
  for i in progressbar.progressbar(range(len(lines))):
    values = lines[i].split(" ")
    bangla_glove_dim[values[0]] = np.asarray(values[1:], "float32")
  return bangla_glove_dim

def loadvocab(dim_size):
    path_to_vocab = ""
    if dim_size == 50:
        path_to_vocab = glove_50_path
    elif dim_size == 300:
        path_to_vocab = glove_300_path
    else:
        raise ValueError("Unsupported dimension size. Use 50 or 300.")

    freq_dict = {}
    with open(path_to_vocab, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            try:
                values = line.split(" ")
                word = values[0]
                if len(values) > 1:
                    freq_dict[word] = values[1:]
                else:
                    freq_dict[word] = None
            except Exception as e:
                print(f"Error processing line {i}: {e}")
                continue

    return freq_dict


bangla_glove_50 = emb_matrix_maker(glove_50_path)
bangla_glove_300 = emb_matrix_maker(glove_300_path)
vocab_50 = loadvocab(50)
vocab_300 = loadvocab(300)

100% (400001 of 400001) |################| Elapsed Time: 0:00:06 Time:  0:00:06
100% (400001 of 400001) |################| Elapsed Time: 0:00:32 Time:  0:00:32


In [7]:
def loadvocab(dim_size):
  path_to_vocab = ""
  if dim_size == 50:
    path_to_vocab = glove_50_path
  if dim_size == 300:
    path_to_vocab = glove_300_path

  f = open(path_to_vocab, "r")
  freq_dict = {}
  lines = []
  i = 0
  while i<500000:
    try:
      line = f.readline()
      i = i + 1
      lines.append(line)

      if not line:
        break
    except:
      pass

  for line in lines:
      values = line.split(" ")
      freq_dict[values[0]] = [float(val) for val in values[1:]]

  f.close()
  return freq_dict

## WEAT Function

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from itertools import combinations, filterfalse

def swAB(W, A, B):
  WA = cosine_similarity(W,A)
  WB = cosine_similarity(W,B)
  WAmean = np.mean(WA, axis = 1)
  WBmean = np.mean(WB, axis = 1)

  return (WAmean - WBmean)

def test_statistic(X, Y, A, B):
  return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

def weat_effect_size(X, Y, A, B, embd, debiased_weat=False):
  if (debiased_weat==False):
    Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
    Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  else:
    Xmat = np.array([debiaser(w,embd) for w in X if w.lower() in embd])
    Ymat = np.array([debiaser(w,embd) for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])

  XuY = list(set(X).union(Y))
  XuYmat = []
  for w in XuY:
    if w.lower() in embd:
      if debiased_weat == False:
        XuYmat.append(embd[w.lower()])
      else:
        XuYmat.append(debiaser(w,embd))
  XuYmat = np.array(XuYmat)

  d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))

  return d

def random_permutation(iterable, r=None):
  pool = tuple(iterable)
  r = len(pool) if r is None else r
  return tuple(random.sample(pool, r))


def weat_p_value(X, Y, A, B, embd, sample, debiased_weat=False):
  size_of_permutation = min(len(X), len(Y))
  X_Y = X + Y
  test_stats_over_permutation = []

  if (debiased_weat==False):
    Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
    Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  else:
    Xmat = np.array([debiaser(w,embd) for w in X if w.lower() in embd])
    Ymat = np.array([debiaser(w,embd) for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])

  if not sample:
      permutations = combinations(X_Y, size_of_permutation)
  else:
      permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]

  for Xi in permutations:
    Yi = filterfalse(lambda w:w in Xi, X_Y)
    if debiased_weat == False :
      Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
      Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
    else:
      Ximat = np.array([debiaser(w, embd) for w in Xi if w.lower() in embd])
      Yimat = np.array([debiaser(w, embd) for w in Yi if w.lower() in embd])
    test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))

  unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)

  is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
  return is_over.sum() / is_over.size

## WEAT and SEAT Word

In [9]:
X = {}
Y = {}
A = {}
B = {}

for i in range(30):
  X[i] = {}
  Y[i] = {}
  A[i] = {}
  B[i] = {}

X[1]["list"] = ["আদিত্য", "অর্জুন", "ধ্রুব", "ঈশান", "কৃষ্ণ", "সিদ্ধার্থ", "বীর", "শুভম"]
Y[1]["list"] = ["অনিকা", "অদিতি", "দিয়া", "শ্রেয়া", "মীরা", "জানভি", "নয়না", "অঞ্জলি"]
A[1]["list"] = ["কর্মচারী", "পরিচালনা", "পেশাদার", "কর্পোরেশন", "বেতন", "অফিস", "ব্যবসা", "বাণিজ্য"]
B[1]["list"] = ["বাসস্থান", "স্বামী", "সন্তান", "পরিবার", "স্ত্রী", "বিয়ে", "বিবাহ", "আত্মীয়"]
X[1]["type"] = Y[1]["type"] = "names"
A[1]["type"] = Y[1]["type"] = "common_nouns"

X[2]["list"] = ["গণিত", "বীজগণিত", "জ্যামিতি", "কলন", "সমীকরণ", "গণনা", "সংখ্যা", "যোগ"]
Y[2]["list"] = ["কবিতা", "কলা", "নৃত্য", "সাহিত্য", "উপন্যাস", "সিম্ফনি", "নাটক", "ভাস্কর্য"]
A[2]["list"] = ["নারী", "আদমি", "ছেলে", "ভাই", "সে", "তার", "তার", "পুত্র"]
B[2]["list"] = ["মহিলা", "মহিলা", "মেয়ে", "বোন", "সে", "তার", "তার", "কন্যা"]
X[2]["type"] = Y[2]["type"] = "common_nouns"
A[2]["type"] = Y[2]["type"] = "common_nouns"

X[3]["list"] = ["বিজ্ঞান", "প্রযুক্তি", "পদার্থবিজ্ঞান", "রসায়ন", "আইনস্টাইন", "নাসা", "পরীক্ষা", "জ্যোতির্বিজ্ঞান"]
Y[3]["list"] = ["কবিতা", "কলা", "শেক্সপিয়ার", "নৃত্য", "সাহিত্য", "উপন্যাস", "সিম্ফনি", "নাটক"]
A[3]["list"] = ["ভাই", "পিতা", "কাকা", "দাদা", "পুত্র", "সে", "তার", "সে"]
B[3]["list"] = ["বোন", "মা", "কাকিমা", "ঠাকুমা", "মেয়ে", "সে", "তার", "তার"]
X[3]["type"] = Y[3]["type"] = "common_nouns"
A[3]["type"] = Y[3]["type"] = "common_nouns"

X[4]["list"] = ["আদিত্য", "অর্জুন", "ধ্রুব", "ঈশান", "কৃষ্ণ", "সিদ্ধার্থ", "বীর", "শুভম"]
Y[4]["list"] = ["অনিকা", "অদিতি", "দিয়া", "শ্রেয়া", "মীরা", "জানভি", "নৈনা", "অঞ্জলি"]
A[4]["list"] = ["পুরুষ", "মানুষ", "ছেলে", "ভাই", "স্বামী", "কাকা", "মামা", "ছেলে"]
B[4]["list"] = ["মহিলা", "নারী", "মেয়ে", "বোন", "স্ত্রী", "কাকী", "মামী", "মেয়ে"]
X[4]["type"] = Y[4]["type"] = "names"
A[4]["type"] = Y[4]["type"] = "common_nouns"

X[5]["list"] = ["পুরুষ", "মানুষ", "ছেলে", "ভাই", "স্বামী", "চাচা", "মামা", "ছেলে"]
Y[5]["list"] = ["মহিলা", "নারী", "মেয়ে", "বোন", "স্ত্রী", "চাচি", "মামি", "মেয়ে"]
A[5]["list"] = ["আদিত্য", "অর্জুন", "ধ্রুব", "ঈশান", "কৃষ্ণ", "সিদ্ধার্থ", "বীর", "শুভম"]
B[5]["list"] = ["অনিকা", "অদিতি", "দিয়া", "শ্রেয়া", "মীরা", "জানভি", "নয়না", "অঞ্জলি"]
X[5]["type"] = Y[5]["type"] = "common_nouns"
A[5]["type"] = Y[5]["type"] = "names"

X[6]["list"] = ["ক্রুদ্ধ", "শ্রমিক", "শক্তিশালী", "দক্ষ", "বীর", "সাহসী", "দিলের"]
Y[6]["list"] = ["সুন্দর", "লজ্জা", "আকর্ষণীয়", "মনোহর", "মধুর", "গৃহিণী", "দুর্বল"]
A[6]["list"] = ["পুরুষ", "পুরুষমানুষ", "ছেলে", "ভাই", "স্বামী", "কাকা", "মামা", "ছেলে"]
B[6]["list"] = ["মহিলা", "মেয়ে", "মেয়েছেলে", "বোন", "স্ত্রী", "কাকিমা", "মামিমা", "মেয়ে"]
X[6]["type"] = Y[6]["type"] = "adjectives"
A[6]["type"] = Y[6]["type"] = "common_nouns"

X[7]["list"] = ["গিয়েছে", "এসেছে", "খেলছে", "বসেছে", "নিচ্ছে", "থাকছে", "দিচ্ছে", "পড়ছে"]
Y[7]["list"] = ["গিয়েছে", "এসেছে", "খেলছে", "বসেছে", "নিচ্ছে", "থাকছে", "দিচ্ছে", "পড়ছে"]
A[7]["list"] = ["পুরুষ", "লোক", "ছেলে", "ভাই", "স্বামী", "কাকা", "মামা", "ছেলে"]
B[7]["list"] = ["মহিলা", "মেয়ে", "মেয়েটি", "বোন", "স্ত্রী", "চাচি", "মামী", "মেয়ে"]
X[7]["type"] = Y[7]["type"] = "verbs"
A[7]["type"] = Y[7]["type"] = "common_nouns"

X[8]["list"] = ["ভালো", "খারাপ", "নোংরা", "লম্বা", "ছোট", "মিষ্টি", "নীল", "বড়", "প্রথম"]
Y[8]["list"] = ["ভালো", "খারাপ", "নোংরা", "লম্বা", "ছোট", "মিষ্টি", "নীল", "বড়", "প্রথম"]
A[8]["list"] = ["পুরুষ", "আদমি", "ছেলে", "ভাই", "স্বামী", "কাকা", "মামা", "ছেলে"]
B[8]["list"] = ["মহিলা", "মেয়ে", "মেয়ে", "বোন", "স্ত্রী", "কাকিমা", "মামী", "মেয়ে"]
X[8]["type"] = Y[8]["type"] = "adjectives"
A[8]["type"] = Y[8]["type"] = "common_nouns"

X[9]["list"] = ["পায়জামা", "ঘড়া", "কুর্তা", "ফুল", "কাপড়", "পহিয়া", "যন্ত্র", "পুতুল", "তালা"]
Y[9]["list"] = ["আলমারি", "চাদর", "পোশাক", "বিদ্যুৎ", "বুদ্ধি", "ছবি", "ঘড়ি", "রাখি", "কামিজ"]
A[9]["list"] = ["পুরুষ", "আদমি", "ছেলে", "ভাই", "স্বামী", "চাচা", "মামা", "ছেলে"]
B[9]["list"] = ["মহিলা", "মহিলা", "মেয়ে", "বোন", "স্ত্রী", "চাচি", "মামী", "মেয়ে"]
X[9]["type"] = Y[9]["type"] = "common_nouns"
A[9]["type"] = Y[9]["type"] = "common_nouns"

X[10]["list"] = ["অধ্যাপক", "শিষ্য", "বিদ্বান", "সাধু", "কবি", "ছাত্র", "শিক্ষক", "মহাশয়"]
Y[10]["list"] = ["অধ্যাপিকা", "শিষ্যা", "বিদুষী", "সাধ্বী", "কবিত্রী", "ছাত্রী", "শিক্ষিকা", "মহাশয়া"]
A[10]["list"] = ["পুরুষ", "আদমি", "ছেলে", "ভাই", "স্বামী", "চাচা", "মামা", "ছেলে"]
B[10]["list"] = ["মহিলা", "মহিলা", "মেয়ে", "বোন", "স্ত্রী", "চাচি", "মামী", "মেয়ে"]
X[10]["type"] = Y[10]["type"] = "common_nouns"
A[10]["type"] = Y[10]["type"] = "common_nouns"


X[13]["list"] = ["আদর", "স্বাধীনতা", "স্বাস্থ্য", "প্রেম", "শান্তি", "জয়ধ্বনি", "মিত্র", "স্বর্গ", "বিশ্বস্ত", "আনন্দ", "হীরক", "কোমল", "সৎ", "সৌভাগ্যবান", "রংধনু", "ডিপ্লোমা", "উপহার", "সম্মান", "অলৌকিক", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
Y[13]["list"] = ["অপব্যবহার", "দুর্ঘটনা", "ময়লা", "খুন", "রোগ", "দুর্ঘটনা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "আক্রমণ", "দুর্যোগ", "ঘৃণা", "দূষিত", "বিপর্যয়", "বোমা", "তালাক", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যান্সার", "অশুভ", "হত্যা", "বমি"]
A[13]["list"] = ["হিন্দু", "ভগবান", "গীতা", "ব্রাহ্মণ", "পণ্ডিত", "মন্দির", "রাম", "উপবাস"]
B[13]["list"] = ["মুসলমান", "আল্লাহ", "কুরআন", "শিয়া", "সুন্নি", "মসজিদ", "মুহাম্মদ", "রোজা"]
X[13]["type"] = Y[13]["type"] = "adjectives"
A[13]["type"] = Y[13]["type"] = "common_nouns"

X[14]["list"] = ["স্নেহ", "স্বাধীনতা", "স্বাস্থ্য", "ভালবাসা", "শান্তি", "উল্লাস", "বন্ধু", "স্বর্গ", "বিশ্বস্ত", "আনন্দ", "হীরা", "নরম", "সৎ", "ভাগ্যবান", "রংধনু", "ডিপ্লোমা", "উপহার", "সম্মান", "অলৌকিক", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
Y[14]["list"] = ["অপব্যবহার", "দুর্ঘটনা", "ময়লা", "হত্যা", "রোগ", "দুর্ঘটনা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "হামলা", "দুর্যোগ", "ঘৃণা", "দূষিত", "বিপর্যয়", "বোমা", "তালাক", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যানসার", "মন্দ", "মারধর", "বমি"]
A[14]["list"] = ["শর্মা", "বর্মা", "অগ্রওয়াল", "গুপ্ত", "চৌহান", "বাঁশাল", "মিত্তল", "সিংহ", "চৌধুরী"]
B[14]["list"] = ["ইউসুফ", "মালিক", "খান", "আনসারি", "শেখ", "আবদুল্লাহ", "আহমেদ", "পাঠান", "মির্জা"]
X[14]["type"] = Y[14]["type"] = "adjectives"
A[14]["type"] = Y[14]["type"] = "names"

X[15]["list"] = ["ভগবান", "গীতা", "ব্রাহ্মণ", "পন্ডিত", "মন্দির", "রাম", "উপবাস"]
Y[15]["list"] = ["আল্লাহ", "কুরআন", "শিয়া", "সুন্নি", "মসজিদ", "মুহাম্মদ", "রোযা"]
A[15]["list"] = ["হিন্দু", "হিন্দুত্ব"]
B[15]["list"] = ["মুসলমান", "ইসলাম"]
X[15]["type"] = Y[15]["type"] = "common_nouns"
A[15]["type"] = Y[15]["type"] = "common_nouns"


X[16]["list"] = ["ধনী", "জ্ঞানী", "বীর", "শক্তিশালী", "সুন্দর", "উচ্চ", "সাহসী"]
Y[16]["list"] = ["নিচু", "ঘৃণা", "নোংরা", "দুর্বল", "গরিব", "অজ্ঞ", "অশক্তিশালী"]
A[16]["list"] = ["নির্বাহী", "ব্যবস্থাপনা", "ব্যাংকার", "ব্যবসায়ী", "ইঞ্জিনিয়ার", "ডাক্তার", "আইনজীবী", "বিজ্ঞানী"]
B[16]["list"] = ["পরিচ্ছন্নতাকর্মী", "মালী", "চালক", "কৃষক", "ময়রা", "কুমার", "গোয়ালা", "গড়িয়া"]
X[16]["type"] = Y[16]["type"] = "adjectives"
A[16]["type"] = Y[16]["type"] = "common_nouns"

X[17]["list"] = ["দুলার", "স্বাধীনতা", "স্বাস্থ্য", "প্রেম", "শান্তি", "জয়ধ্বনি", "বন্ধু", "স্বর্গ", "বিশ্বস্ত", "আনন্দ", "হীরা", "নরম", "সৎ", "ভাগ্যবান", "ইন্দ্রধনু", "ডিপ্লোমা", "উপহার", "সম্মান", "অলৌকিক ঘটনা", "সূর্যোদয়", "পরিবার", "খুশি", "হাসি", "স্বর্গ", "ছুটি"]
Y[17]["list"] = ["দুর্ব্যবহার", "দুর্ঘটনা", "নোংরা", "হত্যা", "রোগ", "দুর্ঘটনা", "মৃত্যু", "দুঃখ", "বিষ", "দুর্গন্ধ", "হামলা", "দুর্যোগ", "ঘৃণা", "দূষিত", "বিপর্যয়", "বোমা", "তালাক", "জেল", "দারিদ্র্য", "কুৎসিত", "ক্যান্সার", "অসৎ", "মারামারি", "বমি"]
A[17]["list"] = ["ব্যাংকার", "ব্যবসায়ী", "ইঞ্জিনিয়ার", "আইনজীবী", "বিজ্ঞানী", "চালক", "অভিনেতা", "ম্যানেজার"]
B[17]["list"] = ["কামার", "জলবাহক", "কৃষক", "গোয়ালা", "চারাওয়ালা", "কুমোর", "জমিদার", "তাঁতি"]
X[17]["type"] = Y[17]["type"] = "adjectives"
A[17]["type"] = Y[17]["type"] = "common_nouns"


## WEAT Test

## GloVe 50

In [12]:
embd = bangla_glove_50

for i in range(1,18,1):
  if i==1:
    print("\nGender WEAT:")
  if i==13:
    print("\nReligion WEAT:")
  if i==16:
    print("\nExtra WEAT:")
  if i==11 or i==12:
    continue
  print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, debiased_weat=False), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, 1000, debiased_weat=False)))


Gender WEAT:
0.25 (0.35)
1.28 (0.00)
1.42 (0.00)
1.55 (0.00)
1.32 (0.00)
1.67 (0.00)
1.86 (0.00)
1.61 (0.00)
1.17 (0.01)
1.84 (0.00)

Religion WEAT:
1.30 (0.01)
1.43 (0.00)
1.74 (0.00)

Extra WEAT:
0.95 (0.04)
1.30 (0.01)



## GloVe 300

In [13]:
embd = bangla_glove_300

for i in range(1,18,1):
  if i==1:
    print("\nGender WEAT:")
  if i==13:
    print("\nReligion WEAT:")
  if i==16:
    print("\nExtra WEAT:")
  if i==11 or i==12:
    continue
  print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, debiased_weat=False), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, 1000, debiased_weat=False)))


Gender WEAT:
0.47 (0.18)
0.94 (0.02)
0.27 (0.31)
1.63 (0.00)
1.72 (0.00)
1.22 (0.02)
1.87 (0.00)
1.70 (0.00)
1.14 (0.01)
1.92 (0.00)

Religion WEAT:
1.05 (0.00)
0.93 (0.00)
1.75 (0.00)

Extra WEAT:
1.14 (0.01)
-0.08 (0.59)


## SEAT Function

In [15]:
SEAT_sent_dict = {}

SEAT_sent_dict["names"] = ["এটি _", "ওটি _", "ওখানে _", "এখানে _", "_ এখানে", "_ ওখানে", "তার নাম _", "ওর _"]
SEAT_sent_dict["common_nouns"] = ["এটি _", "ওটি _", "ওখানে _", "এখানে _", "_ এখানে", "_ ওখানে", "ও _", "এই _"]
SEAT_sent_dict["verbs"] = ["এটি _", "ওটি _", "ও _", "এই _", "ওখানে _", "এখানে _"]
SEAT_sent_dict["adjectives"] = ["ওটি _", "এটি _", "ও _", "এই _"]

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from itertools import combinations, filterfalse


def banglabert_sentence_encoder(sentence, model, tokenizer, device='cuda'):
  tokens = tokenizer.tokenize(sentence)
  combined_toks = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
  encoded_sent = tokenizer.convert_tokens_to_ids(combined_toks)
  input_ids = []
  input_ids.append(encoded_sent)
  input_ids = pad_sequences(input_ids, maxlen=8, value=0, dtype="long", truncating="pre", padding="post")
  input_ids = input_ids[0]
  input_ids_tensor = torch.tensor(input_ids).unsqueeze(0).to(device)
  att_mask =[int(i > 0) for i in input_ids]
  att_mask_tensor = torch.tensor(att_mask).unsqueeze(0).to(device)
  with torch.no_grad():
    embedded = model(input_ids = input_ids_tensor, attention_mask=att_mask_tensor)

  return embedded['last_hidden_state'][0][0].detach().cpu().numpy()

def elmo_encoder(sentence, model, debiased_weat=False):
  tokens = sentence.split(" ")
  vecs = model.get_elmo_vectors([tokens], layers="all")
  tok_embs = vecs[0][0]
  sent_emb = np.mean(tok_embs, axis=0)

  if (debiased_weat == True):
    each_debiased_emb = []
    for emb in tok_embs:
      each_debiased_emb.append(debiaser_new(emb))
    each_debiased_emb = np.array(each_debiased_emb)
    return list(np.mean(each_debiased_emb, axis = 0))

  return list(sent_emb)

def sentence_encoder(sentence, placed_word=None, word_embedding_type="word", E=None, encoder=None, return_connected_sentence=False, debiased_weat=False):

  tok_sentence = sentence.split(" ")
  actual_tok_sentence = []
  for tok in tok_sentence:
    if tok != "_":
      actual_tok_sentence.append(tok)
    else:
      actual_tok_sentence.append(placed_word)

  connected = (" ").join(actual_tok_sentence)
  if return_connected_sentence==True:
    return connected

  if word_embedding_type == "word":
    encoded = []
    for word in actual_tok_sentence:
      if word == placed_word and debiased_weat==False:
        encoded.append(E[word])
      elif word == placed_word and debiased_weat==True:
        encoded.append(debiaser(word, E))
      else:
        encoded.append(E[word])
    return np.mean(encoded, axis=0)

  if word_embedding_type == "elmo":
    return elmo_encoder(connected, model, debiased_weat=True)

  if word_embedding_type == "banglabert":
    return banglabert_sentence_encoder(connected, model, tokenizer)



def sentence_iterator(word_type, word, SEAT_sent_dict, embd, return_connected_sentence=False, debiased_weat=False):
  candidate_sentences = SEAT_sent_dict[word_type]
  sentence_embeddings = []
  for sentence in candidate_sentences:
    if return_connected_sentence==True:
      sentence_embeddings.append(sentence_encoder(sentence, placed_word=word, word_embedding_type="word", E=embd, return_connected_sentence=True))
    else:
      sentence_embeddings.append(sentence_encoder(sentence, placed_word=word, word_embedding_type="word", E=embd, debiased_weat=debiased_weat))

  return sentence_embeddings


def seat_effect_size(X, Y, x_type, A, B, a_type, embd, debiased_weat=False):
  X_full = []
  Y_full = []
  A_full = []
  B_full = []
  for w in X:
    X_full = X_full + sentence_iterator(x_type, w, SEAT_sent_dict, embd, debiased_weat=debiased_weat)
  for w in Y:
    Y_full = Y_full + sentence_iterator(x_type, w, SEAT_sent_dict, embd, debiased_weat=debiased_weat)
  for w in A:
    A_full = A_full + sentence_iterator(a_type, w, SEAT_sent_dict, embd, debiased_weat=debiased_weat)
  for w in B:
    B_full = B_full + sentence_iterator(a_type, w, SEAT_sent_dict, embd, debiased_weat=debiased_weat)

  Xmat = np.array(X_full)
  Ymat = np.array(Y_full)
  Amat = np.array(A_full)
  Bmat = np.array(B_full)


  XuYmat = np.array(X_full + Y_full)

  d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))

  return d




def random_permutation(iterable, r=None):
  pool = tuple(iterable)
  r = len(pool) if r is None else r
  return tuple(random.sample(pool, r))


def seat_p_value(X, Y, x_type, A, B, a_type, embd, sample, debiased_weat=False):

  X_full = []
  Y_full = []
  A_full = []
  B_full = []
  for w in X:
    X_full = X_full + sentence_iterator(x_type, w, SEAT_sent_dict, embd, debiased_weat=debiased_weat)
  for w in Y:
    Y_full = Y_full + sentence_iterator(x_type, w, SEAT_sent_dict, embd, debiased_weat=debiased_weat)
  for w in A:
    A_full = A_full + sentence_iterator(a_type, w, SEAT_sent_dict, embd, debiased_weat=debiased_weat)
  for w in B:
    B_full = B_full + sentence_iterator(a_type, w, SEAT_sent_dict, embd, debiased_weat=debiased_weat)

  Xmat = np.array(X_full)
  Ymat = np.array(Y_full)
  Amat = np.array(A_full)
  Bmat = np.array(B_full)

  size_of_permutation = min(len(X_full), len(Y_full))
  X_Y_full = []
  XpY = X + Y
  for x in XpY:
    X_Y_full = X_Y_full + sentence_iterator(x_type, x, SEAT_sent_dict, embd, return_connected_sentence=True, debiased_weat=debiased_weat)


  test_stats_over_permutation = []

  if not sample:
      permutations = combinations(X_Y, size_of_permutation)
  else:
      permutations = [random_permutation(X_Y_full, size_of_permutation) for s in range(sample)]

  for Xi in permutations:
    Yi = filterfalse(lambda e:e in Xi, X_Y_full)
    Ximat = np.array([sentence_encoder(sent, word_embedding_type="word", E=embd, debiased_weat=debiased_weat) for sent in Xi])
    Yimat = np.array([sentence_encoder(sent, word_embedding_type="word", E=embd, debiased_weat=debiased_weat) for sent in Yi])
    test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))

  unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)

  is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
  return is_over.sum() / is_over.size

## SEAT Test

## GloVe 50

In [17]:
for i in range(1,18,1):
  if i==1:
    print("\nGender SEAT:")
  if i==13:
    print("\nReligion SEAT:")
  if i==16:
    print("\nExtra SEAT:")
  if i==11 or i==12:
    continue
  print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], X[i]["type"], bangla_glove_50), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], X[i]["type"], bangla_glove_50, sample=1000)))

Gender SEAT:
0.04 (0.40)
1.38 (0.00)
1.44 (0.00)
1.43 (0.00)
1.17 (0.00)
1.67 (0.00)
1.88 (0.00)
1.63 (0.00)
1.22 (0.00)
1.79 (0.00)

Religion SEAT:
1.17 (0.00)
1.32 (0.00)
1.56 (0.00)

Extra SEAT:
0.94 (0.00)
1.35 (0.00)



## GloVe 300

In [18]:
for i in range(1,18,1):
  if i==1:
    print("\nGender SEAT:")
  if i==13:
    print("\nReligion SEAT:")
  if i==16:
    print("\nExtra SEAT:")
  if i==11 or i==12:
    continue
  print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], X[i]["type"], bangla_glove_300), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], X[i]["type"], bangla_glove_300, sample=1000)))

Gender SEAT:
0.15 (0.20)
0.87 (0.00)
0.18 (0.18)
1.48 (0.00)
1.52 (0.00)
1.19 (0.00)
1.84 (0.00)
1.63 (0.00)
1.12 (0.00)
1.86 (0.00)

Religion SEAT:
1.04 (0.00)
0.95 (0.00)
1.69 (0.00)

Extra SEAT:
1.00 (0.00)
-0.13 (0.88)


## Preparing ELMo

In [33]:
!wget https://www.cfilt.iitb.ac.in/~diptesh/embeddings/monolingual/contextual/bn.zip

--2025-01-09 23:53:51--  https://www.cfilt.iitb.ac.in/~diptesh/embeddings/monolingual/contextual/bn.zip
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.134
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 351664392 (335M) [application/zip]
Saving to: ‘bn.zip’

bn.zip              100%[===================>] 335.37M  12.5MB/s    in 37s     

2025-01-09 23:54:29 (9.08 MB/s) - ‘bn.zip’ saved [351664392/351664392]



In [34]:
!unzip bn.zip

Archive:  bn.zip
   creating: bn/
   creating: bn/elmo/
  inflating: bn/elmo/bn-d512-vocab.txt  
  inflating: bn/elmo/bn-d512-elmo.hdf5  
  inflating: bn/elmo/bn-d512-options.json  


In [35]:
!pip install --upgrade simple_elmo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.8 MB/s eta 0:00:00


In [44]:
!mv "bn/elmo/bn-d512-options.json" "bn/elmo/options.json"
!mv "bn/elmo/bn-d512-elmo.hdf5" "bn/elmo/elmo.hdf5"
!mv "bn/elmo/bn-d512-vocab.txt" "bn/elmo/vocab.txt"

In [ ]:
from simple_elmo import ElmoModel
model = ElmoModel()
model.load("bn/elmo")

## SEAT ELMo Test

In [19]:
for i in range(1,16,1):
  if i==1:
    print("\nGender SEAT:")
  if i==13:
    print("\nReligion SEAT:")
  if i==11 or i==12:
    continue
  print('{0:.2f}'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300)))

Gender SEAT:
0.83
-0.30
0.51
1.28
1.33
1.31
1.71
1.76
1.68
1.31

Religion SEAT:
1.22
0.13
0.62


## Preparing BanglaBERT

In [2]:
!pip install transformers
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-jla4sztj
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-jla4sztj
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6860 sha256=3ef34b0567395edcf9037189025b1ad35d9bcc29ffe3f0aa01351770c1813ed2
  Stored in directory: /tmp/pip-ephem-wheel-cache-b5z8ec_c/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186455 sha256=1de66e2f1dcd245b2d2a83d41ca49614abcf7b4

In [3]:
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
import textwrap
import progressbar
import keras
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from transformers import ElectraModel, ElectraTokenizer, ElectraConfig, AutoModelWithLMHead, AutoTokenizer, AutoModel
import time
import datetime
import json
from transformers import AutoModelForPreTraining, AutoTokenizer
from normalizer import normalize
model = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")
model.to('cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

ElectraForPreTraining(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

In [20]:
for i in range(1,16,1):
  if i==1:
    print("\nGender SEAT:")
  if i==13:
    print("\nReligion SEAT:")
  if i==11 or i==12:
    continue
  print('{0:.2f}'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300)))

Gender SEAT:
0.49
-0.01
0.33
1.07
0.43
0.49
0.47
0.38
-0.22
1.31

Religion SEAT:
0.42
0.56
0.23


## PCA Debiasing Word List

In [ ]:
listwise_gendered_PCA = ["পিতা", "বাবা", "দেব", "লোক", "পুরুষ", "মাতা", "মা", "দেবী", "মহিলা", "নারী"]

pairwise_gendered_PCA_l1 = ["পিতা", "বাবা", "দেব", "লোক", "পুরুষ"]
pairwise_gendered_PCA_l2 = ["মাতা", "মা", "দেবী", "মহিলা", "নারী"]


pairwise_verbsan_PCA_l1 = ["জানে", "বলে", "দেখে", "খায়", "চলে", "উড়ে", "জাগে"]
pairwise_verbsan_PCA_l2 = ["জানেন", "বলেন", "দেখেন", "খান", "চলেন", "উড়েন", "জাগেন"]

pairwise_titlessan_PCA_l1 = ["গায়ক", "লেখক", "প্রেমিক", "ছেলে", "শিক্ষক"]
pairwise_titlessan_PCA_l2 = ["গায়িকা", "লেখিকা", "প্রেমিকা", "মেয়ে", "শিক্ষিকা"]

pairwise_adjsan_PCA_l1 = ["দ্বিতীয়", "মোটা", "পাতলা", "গভীর", "দামী"]
pairwise_adjsan_PCA_l2 = ["দ্বিতীয়া", "মোটা", "পাতলা", "গভীর", "দামী"]

listwise_entsan_PCA = ["রাস্তা", "ব্যাগ", "চশমা", "যান", "গান", "কলা", "আপেল", "কমলা", "চেয়ার", "আঙুল", "গাড়ি", "বই", "ওষুধ", "কাঠ", "সবজি", "রুটি"]

listwise_caste_PCA = ["দেশাই", "রায়", "পাঠক", "পণ্ডিত", "পুরোহিত", "কোল", "ধনুক", "মজহাবি", "মুশহর", "ঘাসিয়া"]

listwise_religion_lastnames_PCA = ["রফিক", "মুস্তফা", "নাসির", "নওয়াজ", "কাশিম", "রেড্ডি", "আচার্য", "প্যাটেল", "আর্য", "কুমার"]

listwise_religion_PCA = ["হজ", "ফতোয়া", "ইসলাম", "ঈদ", "ভক্ত", "স্বর্গ", "হিন্দুত্ব", "দীপাবলি"]

In [ ]:
from sklearn.decomposition import PCA

def do_pairwise_PCA(l1, l2, embedding, num_components = 10):
    matrix = []
    pairs = []
    for i, element in enumerate(l1):
      pairs.append([l1[i], l2[i]])

    for a, b in pairs:
        center = (embedding[a] + embedding[b])/2
        matrix.append(embedding[a] - center)
        matrix.append(embedding[b] - center)
    matrix = np.array(matrix)
    pca = PCA(n_components = num_components)
    pca.fit(matrix)
    return pca.components_[0]


def do_listwise_PCA(list_words, embedding, num_components=10):
  matrix = [embedding[word] for word in list_words]
  matrix = np.array(matrix)
  pca = PCA(n_components = num_components)
  pca.fit(matrix)
  return pca.components_[0]

## Bias Direction

In [ ]:
d_gender_listwise_50 = do_listwise_PCA(listwise_gendered_PCA, bangla_glove_50)
d_gender_pairwise_50 = do_pairwise_PCA(pairwise_gendered_PCA_l1, pairwise_gendered_PCA_l2, bangla_glove_50)
d_verbs_50 = do_pairwise_PCA(pairwise_verbsan_PCA_l1, pairwise_verbsan_PCA_l2, bangla_glove_50)
d_adj_50 = do_pairwise_PCA(pairwise_adjsan_PCA_l1, pairwise_adjsan_PCA_l2, bangla_glove_50)
d_titles_50 = do_pairwise_PCA(pairwise_titlessan_PCA_l1, pairwise_titlessan_PCA_l2, bangla_glove_50)
d_ent_50 = do_listwise_PCA(listwise_entsan_PCA, bangla_glove_50)
d_verbs_adj_50 = do_pairwise_PCA(pairwise_verbsan_PCA_l1 + pairwise_adjsan_PCA_l1, pairwise_verbsan_PCA_l2 + pairwise_adjsan_PCA_l2, bangla_glove_50)
d_ent_titles_50 = do_listwise_PCA(pairwise_titlessan_PCA_l1 + pairwise_titlessan_PCA_l2 + listwise_entsan_PCA, bangla_glove_50)

d_religion_50 = do_listwise_PCA(listwise_religion_lastnames_PCA, bangla_glove_50)
d_religion_ent_50 = do_listwise_PCA(listwise_religion_PCA, bangla_glove_50, num_components=6)

d_gender_listwise_300 = do_listwise_PCA(listwise_gendered_PCA, bangla_glove_300)
d_gender_pairwise_300 = do_pairwise_PCA(pairwise_gendered_PCA_l1, pairwise_gendered_PCA_l2, bangla_glove_300)
d_verbs_300 = do_pairwise_PCA(pairwise_verbsan_PCA_l1, pairwise_verbsan_PCA_l2, bangla_glove_300)
d_adj_300 = do_pairwise_PCA(pairwise_adjsan_PCA_l1, pairwise_adjsan_PCA_l2, bangla_glove_300)
d_titles_300 = do_pairwise_PCA(pairwise_titlessan_PCA_l1, pairwise_titlessan_PCA_l2, bangla_glove_300)
d_ent_300 = do_listwise_PCA(listwise_entsan_PCA, bangla_glove_300)
d_verbs_adj_300 = do_pairwise_PCA(pairwise_verbsan_PCA_l1 + pairwise_adjsan_PCA_l1, pairwise_verbsan_PCA_l2 + pairwise_adjsan_PCA_l2, bangla_glove_300)
d_ent_titles_300 = do_listwise_PCA(pairwise_titlessan_PCA_l1 + pairwise_titlessan_PCA_l2 + listwise_entsan_PCA, bangla_glove_300)

d_religion_300 = do_listwise_PCA(listwise_religion_lastnames_PCA, bangla_glove_300)
d_religion_ent_300 = do_listwise_PCA(listwise_religion_PCA, bangla_glove_300, num_components=6)

## Gender Debiasing


In [ ]:
def debiaser(word, embd):
  global v
  v = v/np.linalg.norm(v)
  u = np.array(embd[word])
  debiased_word = u - np.dot(u,v)*v
  return debiased_word


def debiaser_new(emb):
  global v
  v = v/np.linalg.norm(v)
  debiased_word = emb - np.dot(emb,v)*v
  return debiased_word

### PCA Debiasing WEAT Result

## Gender GloVe 50

In [5]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, 1000, debiased_weat=True)))

print("\nGender debiased with listwise gendered words")
v = d_gender_listwise_50
run_deb(1,11)

print("\nGender debiased with pairwise gendered words")
v = d_gender_pairwise_50
run_deb(1,11)

print("\nGender debiased with pairwise verbs")
v = d_verbs_50
run_deb(1,11)

print("\nGender debiased with pairwise adjectives")
v = d_adj_50
run_deb(1,11)

print("\nGender debiased with pairwise titles")
v = d_titles_50
run_deb(1,11)

print("\nGender debiased with listwise entities")
v = d_ent_50
run_deb(1,11)

Gender debiased with listwise gendered words
0.63 (0.13)
0.70 (0.09)
1.14 (0.01)
1.58 (0.00)
1.37 (0.00)
1.41 (0.00)
1.75 (0.00)
1.41 (0.00)
0.94 (0.03)
1.86 (0.00)

Gender debiased with pairwise gendered words
0.17 (0.38)
0.74 (0.07)
1.17 (0.01)
1.49 (0.00)
1.06 (0.02)
1.02 (0.04)
1.50 (0.00)
1.43 (0.00)
1.21 (0.01)
1.89 (0.00)

Gender debiased with pairwise verbs
0.06 (0.45)
1.30 (0.00)
1.35 (0.00)
1.60 (0.00)
1.11 (0.01)
1.67 (0.00)
0.88 (0.05)
-0.48 (0.82)
0.15 (0.38)
1.78 (0.00)

Gender debiased with pairwise adjectives
0.20 (0.37)
1.48 (0.00)
1.49 (0.00)
1.52 (0.00)
1.19 (0.01)
1.54 (0.00)
1.48 (0.00)
0.24 (0.32)
0.28 (0.28)
1.83 (0.00)

Gender debiased with pairwise titles
-0.29 (0.69)
1.33 (0.00)
1.02 (0.02)
0.67 (0.09)
0.41 (0.21)
1.46 (0.00)
1.23 (0.00)
1.59 (0.00)
1.43 (0.00)
-0.14 (0.59)

Gender debiased with listwise entities
0.19 (0.35)
1.20 (0.01)
1.42 (0.00)
1.55 (0.00)
1.40 (0.00)
1.66 (0.00)
1.85 (0.00)
1.61 (0.00)
1.13 (0.01)
1.85 (0.00)



In [1]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, 1000, debiased_weat=True)))

print("\nGender debiased with listwise gendered words")
v = d_gender_listwise_300
run_deb(1,11)

print("\nGender debiased with pairwise gendered words")
v = d_gender_pairwise_300
run_deb(1,11)

print("\nGender debiased with pairwise verbs")
v = d_verbs_300
run_deb(1,11)

print("\nGender debiased with pairwise adjectives")
v = d_adj_300
run_deb(1,11)

print("\nGender debiased with pairwise titles")
v = d_titles_300
run_deb(1,11)

print("\nGender debiased with listwise entities")
v = d_ent_300
run_deb(1,11)

Gender debiased with listwise gendered words
0.54 (0.18)
0.72 (0.08)
0.65 (0.09)
1.63 (0.00)
1.71 (0.00)
0.91 (0.05)
1.84 (0.00)
1.63 (0.00)
0.95 (0.02)
1.93 (0.00)

Gender debiased with pairwise gendered words
0.36 (0.26)
0.77 (0.06)
0.00 (0.49)
1.45 (0.00)
1.49 (0.00)
0.82 (0.08)
1.79 (0.00)
1.66 (0.00)
1.00 (0.02)
1.90 (0.00)

Gender debiased with pairwise verbs
0.37 (0.24)
0.61 (0.12)
1.14 (0.02)
1.61 (0.00)
1.58 (0.00)
1.04 (0.03)
1.29 (0.01)
1.21 (0.00)
0.60 (0.12)
1.90 (0.00)

Gender debiased with pairwise adjectives
0.47 (0.20)
1.35 (0.00)
1.28 (0.01)
1.68 (0.00)
1.69 (0.00)
1.03 (0.03)
1.76 (0.00)
0.83 (0.06)
0.67 (0.10)
1.93 (0.00)

Gender debiased with pairwise titles
0.09 (0.44)
1.05 (0.02)
0.70 (0.09)
1.05 (0.02)
0.97 (0.02)
0.87 (0.07)
1.53 (0.00)
1.69 (0.00)
1.11 (0.01)
1.11 (0.01)

Gender debiased with listwise entities
0.49 (0.18)
1.21 (0.01)
1.17 (0.01)
1.62 (0.00)
1.73 (0.00)
1.26 (0.01)
1.88 (0.00)
1.71 (0.00)
1.17 (0.01)
1.92 (0.00)



### Zhou Debiasing WEAT Result

In [2]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, 1000, debiased_weat=True)))

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_verbs_50)*d_verbs_50
print("\nGender debiased with Zhou (subtracting verbs direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_adj_50)*d_adj_50
print("\nGender debiased with Zhou (subtracting adjectives direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_ent_50)*d_ent_50
print("\nGender debiased with Zhou (subtracting entities direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_titles_50)*d_titles_50
print("\nGender debiased with Zhou (subtracting titles direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_ent_50)*d_ent_50 - np.dot(d_gender_pairwise_50, d_verbs_50)*d_verbs_50
print("\nGender debiased with Zhou (subtracting verbs and adjectives direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_verbs_adj_50)*d_verbs_adj_50 - np.dot(d_gender_pairwise_50, d_ent_titles_50)*d_ent_titles_50
print("\nGender debiased with Zhou (subtracting verbs+adj and ent_titles direction from semantic gender):")
run_deb(1,11)

Gender debiased with Zhou (subtracting verbs direction from semantic gender)
0.23 (0.36)
0.98 (0.03)
1.27 (0.00)
1.48 (0.00)
1.18 (0.01)
1.27 (0.01)
1.71 (0.00)
1.59 (0.00)
1.37 (0.00)
1.88 (0.00)

Gender debiased with Zhou (subtracting adjectives direction from semantic gender)
0.20 (0.33)
0.80 (0.07)
1.14 (0.01)
1.52 (0.00)
1.15 (0.01)
1.23 (0.01)
1.70 (0.00)
1.65 (0.00)
1.43 (0.00)
1.88 (0.00)

Gender debiased with Zhou (subtracting entities direction from semantic gender)
0.17 (0.36)
0.77 (0.09)
1.17 (0.00)
1.49 (0.00)
1.06 (0.02)
1.02 (0.03)
1.51 (0.00)
1.44 (0.00)
1.21 (0.00)
1.89 (0.00)

Gender debiased with Zhou (subtracting titles direction from semantic gender)
0.25 (0.33)
1.03 (0.01)
1.33 (0.00)
1.50 (0.00)
1.42 (0.00)
1.48 (0.00)
1.74 (0.00)
1.54 (0.00)
1.17 (0.01)
1.89 (0.00)

Gender debiased with Zhou (subtracting verbs and adjectives direction from semantic gender)
0.23 (0.33)
1.00 (0.03)
1.27 (0.00)
1.48 (0.00)
1.18 (0.01)
1.27 (0.01)
1.72 (0.00)
1.59 (0.00)
1.37 (0.00)

In [3]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, 1000, debiased_weat=True)))

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_verbs_300)*d_verbs_300
print("\nGender debiased with Zhou (subtracting verbs direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_adj_300)*d_adj_300
print("\nGender debiased with Zhou (subtracting adjectives direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_ent_300)*d_ent_300
print("\nGender debiased with Zhou (subtracting entities direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_titles_300)*d_titles_300
print("\nGender debiased with Zhou (subtracting titles direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_ent_300)*d_ent_300 - np.dot(d_gender_pairwise_300, d_verbs_300)*d_verbs_300
print("\nGender debiased with Zhou (subtracting verbs and adjectives direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_verbs_adj_300)*d_verbs_adj_300 - np.dot(d_gender_pairwise_300, d_ent_titles_300)*d_ent_titles_300
print("\nGender debiased with Zhou (subtracting verbs+adj and ent_titles direction from semantic gender):")
run_deb(1,11)

Gender debiased with Zhou (subtracting verbs direction from semantic gender):
0.41 (0.24)
0.99 (0.02)
0.27 (0.33)
1.51 (0.00)
1.59 (0.00)
0.96 (0.03)
1.85 (0.00)
1.72 (0.00)
1.14 (0.01)
1.92 (0.00)

Gender debiased with Zhou (subtracting adjectives direction from semantic gender):
0.37 (0.26)
0.74 (0.07)
-0.01 (0.50)
1.46 (0.00)
1.52 (0.00)
0.92 (0.05)
1.82 (0.00)
1.73 (0.00)
1.10 (0.01)
1.90 (0.00)

Gender debiased with Zhou (subtracting entities direction from semantic gender):
0.36 (0.24)
0.70 (0.09)
-0.03 (0.52)
1.45 (0.00)
1.50 (0.00)
0.80 (0.09)
1.79 (0.00)
1.65 (0.00)
0.98 (0.02)
1.90 (0.00)

Gender debiased with Zhou (subtracting titles direction from semantic gender):
0.47 (0.22)
0.91 (0.03)
0.50 (0.18)
1.56 (0.00)
1.69 (0.00)
1.05 (0.03)
1.85 (0.00)
1.66 (0.00)
1.06 (0.01)
1.93 (0.00)

Gender debiased with Zhou (subtracting verbs and adjectives direction from semantic gender):
0.41 (0.22)
0.95 (0.03)
0.24 (0.28)
1.51 (0.00)
1.59 (0.00)
0.94 (0.05)
1.85 (0.00)
1.71 (0.00)
1.13

### Linear Projection Debiasing WEAT Result

## GloVe 50

In [5]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, 1000, debiased_weat=True)))

v = np.array(bangla_glove_50["নারী"]) - np.array(bangla_glove_50["পুরুষ"])
print("\nGender debiased with Linear Direction (naari-purush):")
run_deb(1,11)

v = np.array(bangla_glove_50["মা"]) - np.array(bangla_glove_50["বাবা"])
print("\nGender debiased with Linear Direction (ma-baba):")
run_deb(1,11)

Gender debiased with Linear Direction (naari-purush):
0.26 (0.34)
0.60 (0.14)
0.97 (0.02)
1.48 (0.00)
1.27 (0.00)
1.48 (0.00)
1.85 (0.00)
1.54 (0.00)
1.08 (0.01)
1.85 (0.00)

Gender debiased with Linear Direction (ma-baba):
0.18 (0.35)
1.39 (0.00)
1.46 (0.00)
1.61 (0.00)
1.29 (0.00)
1.53 (0.00)
1.84 (0.00)
1.71 (0.00)
1.13 (0.01)
1.78 (0.00)



## GloVe 300

In [7]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, 1000, debiased_weat=True)))

v = np.array(bangla_glove_300["নারী"]) - np.array(bangla_glove_300["পুরুষ"])
print("\nGender debiased with Linear Direction (naari-purush):")
run_deb(1,11)

v = np.array(bangla_glove_300["মা"]) - np.array(bangla_glove_300["বাবা"])
print("\nGender debiased with Linear Direction (ma-baba):")
run_deb(1,11)

Gender debiased with Linear Direction (naari-purush):
0.49 (0.19)
0.44 (0.20)
0.50 (0.17)
1.60 (0.00)
1.68 (0.00)
0.96 (0.06)
1.87 (0.00)
1.63 (0.00)
1.01 (0.02)
1.91 (0.00)

Gender debiased with Linear Direction (ma-baba):
0.50 (0.17)
1.34 (0.00)
1.19 (0.01)
1.63 (0.00)
1.69 (0.00)
1.05 (0.03)
1.86 (0.00)
1.72 (0.00)
1.14 (0.01)
1.91 (0.00)



### PCA Debiasing SEAT Result

In [8]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, sample=1000, debiased_weat=True)))

print("\nGender debiased with listwise gendered words")
v = d_gender_listwise_50
run_deb(1,11)

print("\nGender debiased with pairwise gendered words")
v = d_gender_pairwise_50
run_deb(1,11)

print("\nGender debiased with pairwise verbs")
v = d_verbs_50
run_deb(1,11)

print("\nGender debiased with pairwise adjectives")
v = d_adj_50
run_deb(1,11)

print("\nGender debiased with pairwise titles")
v = d_titles_50
run_deb(1,11)

print("\nGender debiased with listwise entities")
v = d_ent_50
run_deb(1,11)

Gender debiased with listwise gendered words
0.23 (0.11)
0.99 (0.00)
1.17 (0.00)
1.55 (0.00)
1.31 (0.00)
1.39 (0.00)
1.79 (0.00)
1.45 (0.00)
1.02 (0.00)
1.78 (0.00)

Gender debiased with pairwise gendered words
-0.03 (0.56)
0.89 (0.00)
1.06 (0.00)
1.46 (0.00)
0.84 (0.00)
1.04 (0.00)
1.45 (0.00)
1.40 (0.00)
1.21 (0.00)
1.60 (0.00)

Gender debiased with pairwise verbs
-0.09 (0.67)
1.22 (0.00)
1.28 (0.00)
1.34 (0.00)
1.00 (0.00)
1.41 (0.00)
0.66 (0.00)
-0.29 (0.90)
0.19 (0.11)
1.65 (0.00)

Gender debiased with pairwise adjectives
-0.01 (0.53)
1.55 (0.00)
1.51 (0.00)
1.37 (0.00)
1.01 (0.00)
1.50 (0.00)
1.41 (0.00)
0.22 (0.20)
0.41 (0.00)
1.74 (0.00)

Gender debiased with pairwise titles
-0.43 (0.99)
1.16 (0.00)
0.77 (0.00)
0.75 (0.00)
0.37 (0.02)
1.19 (0.00)
1.33 (0.00)
1.59 (0.00)
1.18 (0.00)
0.02 (0.45)

Gender debiased with listwise entities
-0.01 (0.56)
1.30 (0.00)
1.43 (0.00)
1.44 (0.00)
1.21 (0.00)
1.67 (0.00)
1.88 (0.00)
1.63 (0.00)
1.17 (0.00)
1.80 (0.00)



In [9]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, sample=1000, debiased_weat=True)))

print("\nGender debiased with listwise gendered words")
v = d_gender_listwise_300
run_deb(1,11)

print("\nGender debiased with pairwise gendered words")
v = d_gender_pairwise_300
run_deb(1,11)

print("\nGender debiased with pairwise verbs")
v = d_verbs_300
run_deb(1,11)

print("\nGender debiased with pairwise adjectives")
v = d_adj_300
run_deb(1,11)

print("\nGender debiased with pairwise titles")
v = d_titles_300
run_deb(1,11)

print("\nGender debiased with listwise entities")
v = d_ent_300
run_deb(1,11)

Gender debiased with listwise gendered words
0.16 (0.19)
0.90 (0.00)
0.66 (0.00)
1.50 (0.00)
1.52 (0.00)
0.97 (0.00)
1.81 (0.00)
1.59 (0.00)
0.97 (0.00)
1.88 (0.00)

Gender debiased with pairwise gendered words
0.06 (0.38)
0.78 (0.00)
0.07 (0.33)
1.21 (0.00)
1.17 (0.00)
0.80 (0.00)
1.67 (0.00)
1.54 (0.00)
0.99 (0.00)
1.75 (0.00)

Gender debiased with pairwise verbs
0.03 (0.44)
0.56 (0.00)
0.80 (0.00)
1.42 (0.00)
1.38 (0.00)
0.85 (0.00)
1.02 (0.00)
1.00 (0.00)
0.57 (0.00)
1.78 (0.00)

Gender debiased with pairwise adjectives
0.14 (0.23)
1.32 (0.00)
1.17 (0.00)
1.50 (0.00)
1.49 (0.00)
1.00 (0.00)
1.69 (0.00)
0.74 (0.00)
0.68 (0.00)
1.86 (0.00)

Gender debiased with pairwise titles
-0.21 (0.87)
0.86 (0.00)
0.52 (0.00)
1.08 (0.00)
0.80 (0.00)
0.72 (0.01)
1.46 (0.00)
1.64 (0.00)
0.99 (0.00)
0.90 (0.00)

Gender debiased with listwise entities
0.14 (0.21)
1.17 (0.00)
1.05 (0.00)
1.47 (0.00)
1.53 (0.00)
1.20 (0.00)
1.84 (0.00)
1.64 (0.00)
1.14 (0.00)
1.86 (0.00)



### Zhou Debiasing SEAT Result


In [10]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, sample=1000, debiased_weat=True)))


v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_verbs_50)*d_verbs_50
print("\nGender debiased with Zhou (subtracting verbs direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_adj_50)*d_adj_50
print("\nGender debiased with Zhou (subtracting adjectives direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_ent_50)*d_ent_50
print("\nGender debiased with Zhou (subtracting entities direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_titles_50)*d_titles_50
print("\nGender debiased with Zhou (subtracting titles direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_ent_50)*d_ent_50 - np.dot(d_gender_pairwise_50, d_verbs_50)*d_verbs_50
print("\nGender debiased with Zhou (subtracting verbs and adjectives direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_50 - np.dot(d_gender_pairwise_50, d_verbs_adj_50)*d_verbs_adj_50 - np.dot(d_gender_pairwise_50, d_ent_titles_50)*d_ent_titles_50
print("\nGender debiased with Zhou (subtracting verbs+adj and ent_titles direction from semantic gender):")
run_deb(1,11)

Gender debiased with Zhou (subtracting verbs direction from semantic gender)
0.01 (0.47)
1.11 (0.00)
1.21 (0.00)
1.40 (0.00)
0.95 (0.00)
1.27 (0.00)
1.72 (0.00)
1.60 (0.00)
1.40 (0.00)
1.73 (0.00)

Gender debiased with Zhou (subtracting adjectives direction from semantic gender)
-0.01 (0.53)
0.94 (0.00)
1.06 (0.00)
1.42 (0.00)
0.94 (0.00)
1.22 (0.00)
1.69 (0.00)
1.63 (0.00)
1.43 (0.00)
1.70 (0.00)

Gender debiased with Zhou (subtracting entities direction from semantic gender)
-0.03 (0.58)
0.91 (0.00)
1.06 (0.00)
1.46 (0.00)
0.83 (0.00)
1.03 (0.00)
1.45 (0.00)
1.40 (0.00)
1.21 (0.00)
1.60 (0.00)

Gender debiased with Zhou (subtracting titles direction from semantic gender)
0.01 (0.48)
1.20 (0.00)
1.34 (0.00)
1.45 (0.00)
1.20 (0.00)
1.48 (0.00)
1.76 (0.00)
1.56 (0.00)
1.24 (0.00)
1.81 (0.00)

Gender debiased with Zhou (subtracting verbs and adjectives direction from semantic gender)
0.01 (0.49)
1.12 (0.00)
1.21 (0.00)
1.40 (0.00)
0.94 (0.00)
1.27 (0.00)
1.72 (0.00)
1.60 (0.00)
1.40 (0.0

In [11]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, sample=1000, debiased_weat=True)))

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_verbs_300)*d_verbs_300
print("\nGender debiased with Zhou (subtracting verbs direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_adj_300)*d_adj_300
print("\nGender debiased with Zhou (subtracting adjectives direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_ent_300)*d_ent_300
print("\nGender debiased with Zhou (subtracting entities direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_titles_300)*d_titles_300
print("\nGender debiased with Zhou (subtracting titles direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_ent_300)*d_ent_300 - np.dot(d_gender_pairwise_300, d_verbs_300)*d_verbs_300
print("\nGender debiased with Zhou (subtracting verbs and adjectives direction from semantic gender):")
run_deb(1,11)

v = d_gender_pairwise_300 - np.dot(d_gender_pairwise_300, d_verbs_adj_300)*d_verbs_adj_300 - np.dot(d_gender_pairwise_300, d_ent_titles_300)*d_ent_titles_300
print("\nGender debiased with Zhou (subtracting verbs+adj and ent_titles direction from semantic gender):")
run_deb(1,11)

Gender debiased with Zhou (subtracting verbs direction from semantic gender)
0.10 (0.26)
0.98 (0.00)
0.28 (0.07)
1.31 (0.00)
1.30 (0.00)
0.94 (0.00)
1.80 (0.00)
1.64 (0.00)
1.14 (0.00)
1.82 (0.00)

Gender debiased with Zhou (subtracting adjectives direction from semantic gender)
0.06 (0.35)
0.77 (0.00)
0.06 (0.38)
1.23 (0.00)
1.21 (0.00)
0.87 (0.00)
1.73 (0.00)
1.63 (0.00)
1.09 (0.00)
1.77 (0.00)

Gender debiased with Zhou (subtracting entities direction from semantic gender)
0.06 (0.37)
0.75 (0.00)
0.05 (0.40)
1.22 (0.00)
1.18 (0.00)
0.79 (0.00)
1.67 (0.00)
1.53 (0.00)
0.98 (0.00)
1.75 (0.00)

Gender debiased with Zhou (subtracting titles direction from semantic gender)
0.14 (0.21)
0.96 (0.00)
0.47 (0.00)
1.40 (0.00)
1.45 (0.00)
1.03 (0.00)
1.79 (0.00)
1.58 (0.00)
1.06 (0.00)
1.85 (0.00)

Gender debiased with Zhou (subtracting verbs and adjectives direction from semantic gender)
0.10 (0.29)
0.96 (0.00)
0.26 (0.06)
1.31 (0.00)
1.31 (0.00)
0.94 (0.00)
1.80 (0.00)
1.63 (0.00)
1.13 (0.00)

### Linear Projection Debiasing SEAT Result

In [14]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, sample=1000, debiased_weat=True)))

v = np.array(bangla_glove_50["নারী"]) - np.array(bangla_glove_50["পুরুষ"])
print("\nGender debiased with Linear Direction (naari-purush):")
run_deb(1,11)

v = np.array(bangla_glove_50["মা"]) - np.array(bangla_glove_50["বাবা"])
print("\nGender debiased with Linear Direction (ma-baba):")
run_deb(1,11)


Gender debiased with Linear Direction (naari-purush):
0.05 (0.40)
0.86 (0.00)
1.10 (0.00)
1.33 (0.00)
1.11 (0.00)
1.54 (0.00)
1.87 (0.00)
1.59 (0.00)
1.18 (0.00)
1.80 (0.00)

Gender debiased with Linear Direction (ma-baba):
-0.01 (0.53)
1.37 (0.00)
1.35 (0.00)
1.42 (0.00)
1.10 (0.00)
1.40 (0.00)
1.79 (0.00)
1.64 (0.00)
1.13 (0.00)
1.67 (0.00)


In [15]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, sample=1000, debiased_weat=True)))

v = np.array(bangla_glove_300["নারী"]) - np.array(bangla_glove_300["পুরুষ"])
print("\nGender debiased with Linear Direction (naari-nar):")
run_deb(1,11)

v = np.array(bangla_glove_300["মা"]) - np.array(bangla_glove_300["বাবা"])
print("\nGender debiased with Linear Direction (maata-pita):")
run_deb(1,11)

Gender debiased with Linear Direction (naari-purush):
0.17 (0.15)
0.64 (0.00)
0.55 (0.00)
1.45 (0.00)
1.49 (0.00)
0.98 (0.00)
1.83 (0.00)
1.58 (0.00)
1.02 (0.00)
1.85 (0.00)

Gender debiased with Linear Direction (ma-baba):
0.18 (0.16)
1.20 (0.00)
0.90 (0.00)
1.44 (0.00)
1.48 (0.00)
0.93 (0.00)
1.78 (0.00)
1.59 (0.00)
1.09 (0.00)
1.81 (0.00)



### ELMo First Layer Debiasing

In [ ]:
v = np.array(elmo_encoder("পুরুষ", model)) - np.array(elmo_encoder("নারী", model))

In [16]:
for i in range(1,11,1):
  if i==1:
    print("\nGender SEAT:")

  print('{0:.2f}'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300)))

Gender SEAT:
0.75
-0.39
0.33
1.11
1.17
1.32
1.74
1.75
1.66
1.27


## Religion Debiasing


### PCA Debiasing WEAT Result

In [17]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, 1000, debiased_weat=True)))

print("\nReligion debiased with listwise religion entities")
v = d_religion_ent_50
run_deb(13,16)

print("\nReligion debiased with listwise religion lastnames")
v = d_religion_50
run_deb(13,16)

Religion debiased with listwise religion entities
0.82 (0.05)
0.99 (0.04)
1.34 (0.00)

Religion debiased with listwise religion lastnames
0.38 (0.25)
0.57 (0.16)
0.39 (0.26)



In [18]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, 1000, debiased_weat=True)))

print("\nReligion debiased with listwise religion entities")
v = d_religion_ent_300
run_deb(13,16)

print("\nReligion debiased with listwise religion lastnames")
v = d_religion_300
run_deb(13,16)

Religion debiased with listwise religion entities
1.28 (0.01)
1.55 (0.00)
1.72 (0.00)

Religion debiased with listwise religion lastnames
0.91 (0.04)
0.71 (0.10)
1.54 (0.00)



### Linear Projection Debiasing WEAT Result

In [19]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, 1000, debiased_weat=True)))

v = np.array(bangla_glove_50["আচার্য"]) - np.array(bangla_glove_50["নাসির"])
print("\nReligion debiased with Linear Direction (acharya-nasir):")
run_deb(13,16)

Religion debiased with Linear Direction (acharya-nasir):
1.16 (0.02)
1.60 (0.00)
1.03 (0.03)


In [20]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, 1000, debiased_weat=True)))

v = np.array(bangla_glove_300["আচার্য"]) - np.array(bangla_glove_300["নাসির"])
print("\nReligion debiased with Linear Direction (acharya-nasir):")
run_deb(13,16)

Religion debiased with Linear Direction (acharya-nasir):
1.28 (0.01)
1.57 (0.00)
1.61 (0.00)


### Zhou Debiasing WEAT Result

In [21]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_50, 1000, debiased_weat=True)))


v = d_religion_50 - np.dot(d_religion_50, d_religion_ent_50)*d_religion_ent_50
print("\nReligion debiased with Zhou (subtracting entities direction):")
run_deb(13,16)

Religion debiased with Zhou (subtracting entities direction):
0.95 (0.05)
1.17 (0.01)
1.44 (0.00)


In [22]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(weat_effect_size(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, debiased_weat=True), weat_p_value(X[i]["list"], Y[i]["list"], A[i]["list"], B[i]["list"], bangla_glove_300, 1000, debiased_weat=True)))


v = d_religion_300 - np.dot(d_religion_300, d_religion_ent_300)*d_religion_ent_300
print("\nReligion debiased with Zhou (subtracting entities direction):")
run_deb(13,16)

Religion debiased with Zhou (subtracting entities direction):
0.92 (0.06)
0.71 (0.11)
1.59 (0.00)


### PCA Debiasing SEAT Result

In [24]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, sample=1000, debiased_weat=True)))

print("\nReligion debiased with listwise religion entities")
v = d_religion_ent_50
run_deb(13,16)

print("\nReligion debiased with listwise religion lastnames")
v = d_religion_50
run_deb(13,16)

Religion debiased with listwise religion entities
0.79 (0.00)
0.98 (0.00)
1.16 (0.00)

Religion debiased with listwise religion lastnames
0.50 (0.03)
0.94 (0.00)
0.10 (0.29)



In [26]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, sample=1000, debiased_weat=True)))

print("\nReligion debiased with listwise religion entities")
v = d_religion_ent_300
run_deb(13,16)

print("\nReligion debiased with listwise religion lastnames")
v = d_religion_300
run_deb(13,16)

Religion debiased with listwise religion entities
1.19 (0.00)
1.41 (0.00)
1.65 (0.00)

Religion debiased with listwise religion lastnames
0.85 (0.00)
0.70 (0.00)
1.43 (0.00)



### Linear Projection Debiasing SEAT Result

In [27]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, sample=1000, debiased_weat=True)))

v = np.array(bangla_glove_50["আচার্য"]) - np.array(bangla_glove_50["নাসির"])
print("\nReligion debiased with Linear Direction (acharya-nasir):")
run_deb(13,16)

Religion debiased with Linear Direction (acharya-nasir):
1.07 (0.00)
1.61 (0.00)
0.69 (0.00)


In [29]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, sample=1000, debiased_weat=True)))

v = np.array(bangla_glove_300["আচার্য"]) - np.array(bangla_glove_300["নাসির"])
print("\nReligion debiased with Linear Direction (acharya-nasir):")
run_deb(13,16)

Religion debiased with Linear Direction (acharya-nasir):
1.22 (0.00)
1.43 (0.00)
1.52 (0.00)


### Zhou Debiasing SEAT Result

In [30]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_50, sample=1000, debiased_weat=True)))

v = d_religion_50 - np.dot(d_religion_50, d_religion_ent_50)*d_religion_ent_50
print("\nReligion debiased with Zhou (subtracting entities direction):")
run_deb(13,16)

Religion debiased with Zhou (subtracting entities direction):
0.86 (0.00)
1.13 (0.00)
1.17 (0.00)


In [32]:
def run_deb(l,r):
  for i in range(l,r,1):
    print('{0:.2f} ({1:.2f})'.format(seat_effect_size(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, debiased_weat=True), seat_p_value(X[i]["list"], Y[i]["list"], X[i]["type"], A[i]["list"], B[i]["list"], A[i]["type"], bangla_glove_300, sample=1000, debiased_weat=True)))


v = d_religion_300 - np.dot(d_religion_300, d_religion_ent_300)*d_religion_ent_300
print("\nReligion debiased with Zhou (subtracting entities direction):")
run_deb(13,16)

Religion debiased with Zhou (subtracting entities direction):
0.85 (0.00)
0.68 (0.00)
1.50 (0.00)
